### Week 1. Titanic Classification with DistilBERT + XGBoost 과제

Week 1. 과제로는 `week1_lecture.ipynb` 코드 예제를 참고하여, `kaggle`에서 titanic 데이터를 받아 데이터 전처리를 진행한 후 학습해 자신의 모델을 만든 후, ipynb 파일과 결과 csv파일을

- ***hw1_학번_이름.ipynb***
- ***submission.csv***

실전데이터사이언스 스터디 repository week1 폴더에 커밋하시면 됩니다.!

https://github.com/a2ran/prac_ds

타이타닉 데이터 전처리 예시는 다음과 같습니다.

1. Filling out missing (NaN) Values
2. VIF으로 분산이 높은 column 제거 or 수정
3. train_test_split 으로 train/val 나누는 과정 수정
4. "bert-base-uncased" 이외 다른 BERT 모델을 사용해 Embedding하기
5. "xgboost" 이외 다른 머신러닝 알고리즘 사용

참고할만한 example은 다음과 같습니다.


https://github.com/minsuk-heo/kaggle-titanic/blob/master/titanic-solution.ipynb

`Optional` :

https://github.com/mrdbourke/your-first-kaggle-submission/blob/master/kaggle-titanic-dataset-example-submission-workflow.ipynb
https://github.com/agconti/kaggle-titanic/blob/master/Titanic.ipynb


# 기본 제공 코드

### 1. Kaggle에서 데이터 받아오기

In [1]:
## pip install ~ : 패키지 다운로드
## -q : 로그 메세지 출력 X

!pip install -q kaggle

In [6]:
## Kaggle에서 데이터를 받아오기 위해서는 Authenticator Token인 "kaggle.json"이 있어야 합니다.
## 자세한 내용은 영상을 참고하세요

from google.colab import drive
drive.mount("/content/drive")

!mkdir ~/.kaggle #코랩 노트북 안에 kaggle이라는 폴더를 만든것임
## Drive에 kaggle.json을 업로드한 경로를 적으시면 됩니다. ex) (/content/drive/MyDrive/study_session/kaggle.json)
!cp /content/drive/MyDrive/DSL_2023-2_실전데이터사이언스방법론/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Mounted at /content/drive
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [7]:
## Titanic 공모전 데이터를 코랩으로 받고, zip 압축 해제하기

# 공모전 이름
competition_name = "titanic"

# 공모전 다운로드 to local environment
! kaggle competitions download -c {competition_name}

# {competition_name}이름의 폴더에 zip 파일 압축해제
! unzip {competition_name + ".zip"} -d {competition_name}

# 드라이브 확인을 완료했으므로 드라이브 mount를 해제합니다.
drive.flush_and_unmount()

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 1.15MB/s]
Archive:  titanic.zip
  inflating: titanic/gender_submission.csv  
  inflating: titanic/test.csv        
  inflating: titanic/train.csv       


### 2. 데이터 전처리

In [4]:
## GPU 활성화
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
import numpy as np
import pandas as pd

train = pd.read_csv('titanic/train.csv')
test = pd.read_csv('titanic/test.csv')

In [29]:
## Your Codes
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [30]:
#결측치 확인
print(train.isnull().sum()) #age, cabin, embarked에 결측치
print('')
print(test.isnull().sum()) #age, fare, cabin에 결측치

# 우리가 맞춰야되는거는 survived

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [59]:
from sklearn.model_selection import train_test_split

X = train
y = train['Survived']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.05, random_state = 77, stratify = y)

In [60]:
#어차피 BERT를 쓸거니까 이걸 텍스트로 바꿔보자.
def make_sentence(dataframe, i): #i means index
  Pclass=str(dataframe.iloc[i,2])
  Name=str(dataframe.iloc[i,3])
  Sex=str(dataframe.iloc[i,4])
  Age=str(dataframe.iloc[i,5])
  SibSp=str(dataframe.iloc[i,6])
  Parch=str(dataframe.iloc[i,7])
  Ticket=str(dataframe.iloc[i,8])
  Fare=str(dataframe.iloc[i,9])
  Cabin=str(dataframe.iloc[i,10])
  Embarked=str(dataframe.iloc[i,11])

  sentence=("This passenger's Pclass is {0}. The name is {1}. The sex is {2} and is {3} years old. It has {4} siblings or spouse aboard "
          "and {5} parents or children aboard. The ticket number is {6}, and the fare paid was {7}. It stayed in cabin {8}, "
          "and its port of embarkation was {9}.".format(Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked))
  return sentence


In [63]:
make_sentence(X_train,0)

"This passenger's Pclass is 3. The name is Elsbury, Mr. William James. The sex is male and is 47.0 years old. It has 0 siblings or spouse aboard and 0 parents or children aboard. The ticket number is A/5 3902, and the fare paid was 7.25. It stayed in cabin nan, and its port of embarkation was S."

In [66]:
train_labels = [int(_) for _ in y_train.values]
texts = [make_sentence(X_train, i) for i in range(len(X_train))]

In [69]:
print(len(texts))
print(texts[0])

846
This passenger's Pclass is 3. The name is Elsbury, Mr. William James. The sex is male and is 47.0 years old. It has 0 siblings or spouse aboard and 0 parents or children aboard. The ticket number is A/5 3902, and the fare paid was 7.25. It stayed in cabin nan, and its port of embarkation was S.


### 3. 데이터 임베딩

In [70]:
from typing import List
from tqdm.notebook import tqdm

!pip install -q transformers
from transformers import AutoModel, AutoTokenizer

class Encode_with_BERT:
    def __init__(self):
        ## Huggingface에서 BERT 모델을 받아옵니다.
        self.tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
        self.model = AutoModel.from_pretrained('bert-base-uncased').to(device)

    ## 입력한 `texts`을 768 길이의 숫자벡터로 특징을 추출합니다.
    def extract(self, texts: List[str]):
        features = np.zeros((len(texts), 768), dtype = np.float16)
        for index, text in enumerate(tqdm(texts)):
            tokenized_text = self.tokenizer(text, return_tensors="pt").to(device)
            model_output = self.model(**tokenized_text)[0].detach().cpu()
            features[index, :] = model_output.numpy().mean(axis=1)

        return features

### 4. 임베딩한 데이터 머신러닝 알고리즘으로 분류

In [71]:
from xgboost.sklearn import XGBClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

extractor = Encode_with_BERT()
scaler = StandardScaler()
classifier = XGBClassifier(use_label_encoder = False)

## XGBoost 학습
train_features = scaler.fit_transform(extractor.extract(texts)) ## to (len(texts), 768) 숫자벡터
classifier.fit(train_features, train_labels) ## XGBoost train

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:1395: UserWarning: `use_label_encoder` is deprecated in 1.7.0.
  warnings.warn("`use_label_encoder` is deprecated in 1.7.0.")


  0%|          | 0/846 [00:00<?, ?it/s]

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [72]:
preds = classifier.predict(scaler.transform(extractor.extract(texts)))
accuracy_score(train_labels, preds) #train accuracy 100

  0%|          | 0/846 [00:00<?, ?it/s]

1.0

In [73]:
## Prediction with Test Data

answer = [int(_) for _ in y_val.values]
texts = [make_sentence(X_val, i) for i in range(len(X_val))]
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

## 모델 예측의 정밀도 측정!
accuracy = accuracy_score(answer, preds)
print(f'\naccuracy : {accuracy*100:.2f}%')

#validation accuracy =84.44%
#생각보다 성능 별로네ㅎㅎ

  0%|          | 0/45 [00:00<?, ?it/s]


accuracy : 84.44%


### 5. csv로 저장 + kaggle에 submit

In [77]:
import csv

ids = test['PassengerId'].values
test.insert(0, 'a', [0 for i in range(len(test))])
texts = [make_sentence(test, i) for i in range(len(test))]

In [79]:
texts[0]

"This passenger's Pclass is 3. The name is Kelly, Mr. James. The sex is male and is 34.5 years old. It has 0 siblings or spouse aboard and 0 parents or children aboard. The ticket number is 330911, and the fare paid was 7.8292. It stayed in cabin nan, and its port of embarkation was Q."

In [80]:
preds = classifier.predict(scaler.transform(extractor.extract(texts)))

with open('submission.csv', "w") as to_file:
    csvwriter = csv.writer(to_file, delimiter=",", quotechar='"')
    csvwriter.writerow(["PassengerId", "Survived"])
    for id, pred in zip(ids, preds):
        csvwriter.writerow([id, pred])

  0%|          | 0/418 [00:00<?, ?it/s]

In [81]:
sub = pd.read_csv('submission.csv')
sub.head(5)

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
